In [1]:
from google.colab import drive
drive.mount("/content/mydrive/")
%cd /content/mydrive/MyDrive/LTSF_Linear

import os
import json
from glob import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import levene, bartlett
from scipy import stats

from _data_provider.data_loader import get_ydata


root_path = "dataset/Water_Week"
dir_list = sorted(os.listdir(root_path))
dir_list

Drive already mounted at /content/mydrive/; to attempt to forcibly remount, call drive.mount("/content/mydrive/", force_remount=True).
/content/mydrive/MyDrive/LTSF_Linear


['C001팔당2취',
 'D001청주정',
 'D002구미',
 'D003고령',
 'D004주암',
 'E003본포',
 'E004남강']

In [2]:
# returns: 각 주차의 평균 feature 값을 담은 리스트
def median_split(dir:str, target:str, feature:str):
    """_summary_
    Args:
        dir (str): _description_
        target (str): _description_
        feature (str): _description_
    Returns:
        list, list
    """
    in_cols, target_cols, df_target = get_ydata(root_path, dir)
    median_target = df_target[target].median()
    index_listG = list(df_target[df_target[target] > median_target].index) 
    index_listL = list(df_target[df_target[target] <= median_target].index) 

    data_list = sorted(glob(os.path.join(root_path, dir, "csv/*.csv")))
    cnt = 0
    path_listG = []
    path_listL = []
    for file in data_list:
        if cnt in index_listG:
            path_listG.append(file)
        elif cnt in index_listL:
            path_listL.append(file)
        cnt += 1
    
    weekly_meanG = []
    for file in path_listG:
        df_ = pd.read_csv(file)
        input_columns = df_.columns.intersection(in_cols)
        x_data = df_[input_columns]
        mean = x_data[feature].mean()
        weekly_meanG.append(mean)

    weekly_meanL = []
    for file in path_listL:
        df_ = pd.read_csv(file)
        input_columns = df_.columns.intersection(in_cols)
        x_data = df_[input_columns]
        mean = x_data[feature].mean()
        weekly_meanL.append(mean)


    return weekly_meanG, weekly_meanL

In [3]:
G, L = median_split(dir_list[1], "2-MIB", "turbidity")
len(G), len(L)

(24, 76)

In [4]:
def ind_t_test(dir:str, target:str, feature:str):
    """_summary_
    Args:
        dir (str): _description_
        target (str): _description_
        feature (str): _description_
    Returns:
        str
    """
    G, L = median_split(dir, target, feature)

    pval1 = levene(G, L)
    pval2 = bartlett(G, L)
    p = 0.05

    equal_var=False
    if pval1.pvalue > p and pval2.pvalue > p:
        equal_var=True

    result = stats.ttest_ind(G, L, equal_var=equal_var)
    output = ''
    if result.pvalue < p:
        output = f"{dir}  {target}  {feature}:  {result}"

    
    return output

In [6]:
for dir in dir_list:
    in_cols, target_cols, df_target = get_ydata(root_path, dir)
    for target in df_target.columns:
        print(ind_t_test(dir, target, "turbidity"))



D001청주정  TOC  turbidity:  Ttest_indResult(statistic=4.051653029156705, pvalue=0.00015542274978364834)
D001청주정  blue-green_algae  turbidity:  Ttest_indResult(statistic=3.398666863361974, pvalue=0.0011415605506067082)

D001청주정  2-MIB  turbidity:  Ttest_indResult(statistic=2.359979340338743, pvalue=0.025798143648121864)
D001청주정  Geosmin  turbidity:  Ttest_indResult(statistic=-4.541892593480467, pvalue=1.976602273457534e-05)

D001청주정  Mn  turbidity:  Ttest_indResult(statistic=2.7402192620065615, pvalue=0.00799327131967927)


D002구미  2-MIB  turbidity:  Ttest_indResult(statistic=2.0128026121591307, pvalue=0.046936155568128454)

D003고령  diatomeae  turbidity:  Ttest_indResult(statistic=-4.089057729036955, pvalue=0.00014554107918547394)
D003고령  2-MIB  turbidity:  Ttest_indResult(statistic=3.1058465926809755, pvalue=0.0028471282421283243)



ValueError: ignored